In [47]:
import spotipy
import spotipy.util as util
import requests
import time #audio analysis
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import csv

In [48]:
cid = '049ade7215e54c63a2b628f3784dc407'
secret = '1ad94ff2aaec4184b8ba80416ee09bc6'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [49]:
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [50]:
#Files
playlist_file = 'Data/playlist_datafile.txt'

#Master List
all_playlists =[]


In [51]:
def get_playlist_tracks(nexturl):
    #Get playlists
    try:
        resp = requests.get(url=nexturl,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
    
    #Generate json
    response = resp.json()
    
        
    #Get limit to set range for loop
    response_limit = response['limit']-1
    
    for x in range(response_limit):
        try:
            uri = response['items'][x]['track']['uri']
            title = response['items'][x]['track']['name']
            all_playlists.append({'uri':uri,'title':title})
        
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
            
    try:    
        if (nexturl is not None):#is not none
            get_playlist_tracks(response['next'])
        else:
            return
    except:
        return

    

In [52]:
#Read file for playlist ID
with open(playlist_file) as csvfile:
    playlist_reader = csv.reader(csvfile,delimiter=':')
    for row in playlist_reader:
        playlist_id_lookup = row[2].strip()
        get_playlist_tracks('https://api.spotify.com/v1/playlists/{p}/tracks'.format(p=playlist_id_lookup))
    

In [53]:
'''
for index,x in enumerate(all_playlists):
    print(index,x)
    for key,value in x.items():
        if value is None:
            print(key,value)
'''


'\nfor index,x in enumerate(all_playlists):\n    print(index,x)\n    for key,value in x.items():\n        if value is None:\n            print(key,value)\n'

In [54]:
pd.DataFrame(all_playlists)

,uri,title
0,spotify:track:62ljuuXo0zrcrtJnnOhfxT,Dear April (Side A - Acoustic)
1,spotify:track:5NRFBtaNmTV0F9NaPwhOL2,Still Here
2,spotify:track:2ca5xMtPpmULDE6aG3VuEn,Tempo
3,spotify:track:7kfl2DwtW29sYrUGRH5B2Y,Memphis
4,spotify:track:3sp4CJbR97KYUnyxup7Zzn,SOLDIERS (feat. Sampha)
...,...,...
3168,spotify:track:1zlbAmFp6m9aWu3x7mTlAC,OCME
3169,spotify:track:5RCQFKVz8F2LwpWi2vHFOl,The Peloton
3170,spotify:track:5Npb23dH7Ky7oJJshW43Zy,Reed Clouds
3171,spotify:track:4QN5BM506dAyQb8zLPvCcT,The Breakaway
